In [ ]:
## Parameters for Pattern Lattice Builder
verbose     = True

## multiprocess
use_mp       = True
if use_mp:
    import os
    n_cores = max(1, round(os.cpu_count()/2) + 1)
else:
    n_cores = 1
print(f"n_cores: {n_cores}")

## set font for Japanese character display
use_multibyte_chars = False
print(f"use_multibyte_chars: {use_multibyte_chars}")

In [ ]:
## select target file
from pathlib import Path
wd = Path("./data3-morphology")
target_files = list(wd.glob('*.txt')) + list(wd.glob('*.csv'))
import pprint as pp
pp.pprint(f"target files: {target_files}")

In [ ]:
## select one target file
target_file  = target_files[-1]
print(f"target file: {target_file}")

In [ ]:
## load data
## remove comment lines
comment_escapes  = [ '#', '%' ]
with open(target_file, 'r') as f:
    raw_lines = [ line.strip() for line in f.readlines() if len(line) > 0 and line[0] not in comment_escapes ]

## remove inline comments
check = True
lines = []
for line in raw_lines:
    body = [ ]
    for char in line:
        if char in comment_escapes:
            break
        else:
            body.append(char)
    t = "".join([ x.strip() for x in body ])
    lines.append(str(t))
print(f"lines: {lines}")

In [ ]:
## lowercase
lines = [ line.lower() for line in lines ]

In [ ]:
## filter lines by regex
pattern_filtering = False
filter_pattern = r'.*g.*'
if pattern_filtering:
    import re
    filtered_lines = [ line for line in lines if re.match(filter_pattern, line) ]
else:
    filtered_lines = lines
print(f"filtered_lines has {len(filtered_lines)} entries")
print(f"filtered_lines: {filtered_lines}")

In [ ]:
## set field separator
if str(target_file).endswith(".csv"):
    field_seps = r",;"
else:
    field_seps = r""
print(f"field_seps: '{field_seps}'")

In [ ]:
## set phrase segmentation parameters
use_larger_segments = True
if use_larger_segments:
    sep2_is_suppressive = True
else:
    sep2_is_suppressive = False
print(f"sep2_is_suppressive: {sep2_is_suppressive}")

In [ ]:
filtered_lines

In [ ]:
## split line into fields
import gPLB as plb
data = [ plb.segment_with_levels(line, seps = field_seps, sep2_is_suppressive = sep2_is_suppressive, split_hyphenation = False, uncapitalize = False, normalize = True) for line in filtered_lines ]
pp.pprint(f"data: {data}")

In [ ]:
## filter lines by length
size_filtering = True
min_length =  2
max_length =  4
if size_filtering:
    data = [ item for item in data if min_length <= len(item) and len(item) <= max_length ]
else:
    data = data
print(f"data has {len(data)} entries")

import pprint as pp
pp.pprint(data)

In [ ]:
## get graph_layout parameters
n_items    = len(data)
max_n_segs = max(map(len, data))
print(f"max_n_segs: {max_n_segs}, n_items: {n_items}")

In [ ]:
## install required packages
#!conda install matplotlib multiprocess networkx -y

In [ ]:
## parameters for PLB
gap_marks      = [ "…", "_" ]
gap_mark       = gap_marks[0]
tracer         = "~"

## if instantiation check is reflexive
reflexive      = True

## if node pooling is reductive
reductive      = True

##
p_metrics = ['rank', 'gap_size', 'size']
p_metric = p_metrics[0]

## if robust Z-score is used instead of normal Z-score
use_robust_zscore   = True

## select the bases of z-scores: link sources or link targets
zscores_from_targets = False

In [ ]:
## settings for font
def setup_font(system_font_dir: str = "/System/Library/Fonts/", # Mac only 
        user_font_dir: str = "/Library/Fonts/", # Mac only
        # use the version installed via TeXLive
        user_font_dir2: str = "/usr/local/texlive/2013/texmf-dist/fonts/truetype/public/ipaex/"
        ):
    import matplotlib
    from matplotlib import font_manager as Font_manager
    if use_multibyte_chars:
        ## select font
        multibyte_font_names = [    "IPAexGothic",  # 0 Multi-platform font
                                    "Hiragino sans" # 1 Mac only
                                ]
        multibyte_font_name  = multibyte_font_names[0]
        ## tell where target fonts are
        if multibyte_font_name == "IPAexGothic":
            try:
                Font_manager.fontManager.addfont(f"{user_font_dir}ipaexg.ttf")
            except FileNotFoundError:
                Font_manager.fontManager.addfont(f"{user_font_dir2}ipaexg.ttf")
        elif multibyte_font_name == "Hiragino sans":
            Font_manager.fontManager.addfont(f"{system_font_dir}ヒラギノ角ゴシック W0.ttc")
        ## check result
        matplotlib.rc('font', family = multibyte_font_name)
    else:
        multibyte_font_name = None
        matplotlib.rcParams['font.family'] = "Sans-serif"
    ##
    print(f"multibyte_font_name: {multibyte_font_name}")
    print(f"matplotlib.rcParams['font.family']: {matplotlib.rcParams['font.family']}")
    ##
    return multibyte_font_name

In [ ]:
#!conda install multiprocess networkx seaborn -y

In [ ]:
## build pattern lattices from data
import gPLB as plb
PLs = [ ]
for item in data:
    ## build a Patter for each item
    pat    = plb.Pattern(item, gap_mark, tracer)
    ## run through generality levels
    for generality in range(0, 3):
        patlat = plb.PatternLattice(pat, reflexive = reflexive, generality = generality)
        if verbose:
            print(patlat)
        PLs.append(patlat)

In [ ]:
## print forms
print_forms = True
seg_joint = ", "
if print_forms:
    print(f"skippy n-grams of the merged lattice")
    for i, patlat in enumerate(PLs):
        for j, node in enumerate(patlat.nodes):
            print(f"patlat{i+1:02d}.node{j+1:03d}: {seg_joint.join(node.get_form())}")

In [ ]:
## graph layout selection
graph_layouts = [   'Multi_partite', # 0
                    'Graphviz', # 1
                    'Fruchterman_Reingold', # 2
                    'ARF', # 3
                    'Spring', # 4
                    'Circular', # 5
                    'Shell', # 6
                    'Spectral', # 7
                    'Kamada_Kawai', # 8
                    'Breadth-First Search', # 9
                    'Planar', # 10
                 ]
#
graph_layout = graph_layouts[0]
print(f"graph_layout: {graph_layout}")

##
MPG_keys = ['gap_size', 'rank']
MPG_key  = MPG_keys[0]
print(f"MPG_key: {MPG_key}")

In [ ]:
#!conda install networkx matplotlib -y 

In [ ]:
#!conda install PyQt5 -y # fails
#!pip install -U PyQt5
#!conda update conda -y

In [ ]:
draw_PL_before_merger = False
multibyte_font_name = setup_font()
if draw_PL_before_merger:
    for PL in PLs:
        PL.draw_lattice (graph_layout, MPG_key, draw_lattice = False, draw_inline = True, auto_figsizing = True, generality = generality, zscores_from_targets = zscores_from_targets, font_name = multibyte_font_name)

In [ ]:
## merge Patern lattices
M = plb.merge_pattern_lattices (PLs, generality = generality, gen_links_internally = True, use_mp = use_mp, reflexive = True, reductive = True, check = False)

In [ ]:
print(f"##Calculating z-scores ...")
plb.gen_zscores_from_targets_by (p_metric, M, gap_mark = gap_mark, tracer = tracer, use_robust_zscore = use_robust_zscore, check = False)
plb.gen_zscores_from_sources_by (p_metric, M, gap_mark = gap_mark, tracer = tracer, use_robust_zscore = use_robust_zscore, check = False)

In [ ]:
#!conda install pydot -y

In [ ]:
## Draw pattern lattice in available layouts
for layout in graph_layouts:
    print(f"graph_layout: {layout}")
    try:
        M.draw_lattice (layout, MPG_key, draw_lattice = True, draw_inline = True, generality = generality, zscores_from_targets = zscores_from_targets, font_name = multibyte_font_name)
    except (Error, NetworkXException) as e:
        print(f"#NetworkXError: {e}")